In [1]:
import pickle
import torch
import model.NPVAE as NPVAE
import easydict 
from collections import OrderedDict
from model.utils import *
from collections import Counter
import argparse
import time
from rdkit import RDLogger, Chem
import datetime
from train import collate as collate
import numpy as np
import warnings

Using backend: pytorch


In [2]:
parser = argparse.ArgumentParser()

args = easydict.EasyDict({
"x_size" : 2048,
"h_size" : 512,
"mid_size" : 256,
"z_dim" : 256,
"label_size" : 4802,
"max_iter" : 500,
"label_path" : "/home/yddr/code/NPVAE/electrolyte_preprocessed/input_data/labels",
"device" : "cuda",
"status" : "test",
"n_trial" : 3,
"test3D" : False,
"return2D": None,
"load_path": "/home/yddr/code/NPVAE/param_data_4",
"load_epoch": 100,
"frequency": 5,
"prepared_path": "/home/yddr/code/NPVAE/electrolyte_preprocessed",
"fpbit": 2048,
"radius": 2
})

In [3]:
with open('/home/yddr/code/NPVAE/electrolyte_preprocessed/input_data/labels', 'rb') as f:
    labels = pickle.load(f)

model = NPVAE.Chem_VAE(2048, 512, 256, 256, 0, len(labels), 500, labels, None, None, None, None, None, 'cuda', "test", 3, True)

load_path = "/home/yddr/code/NPVAE/param_data_4/model.iter-100"
state_dict = torch.load(load_path)
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k.replace('module.', '')
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
model.eval()
model = model.to('cuda')

In [5]:
smiles = "C(CCCC(=O)O)[C@@H]1CCSS1"

# Preproocess 1

mol = Chem.MolFromSmiles(smiles)
Chem.Kekulize(mol)
mol = Chem.RemoveHs(mol)

# Preproocess 2
cl, frag = count_fragments(mol)
count_labels = Counter(cl)

# Preproocess 3
mapidxs, labelmap_dict, bondtypes, max_mapnums, frag = find_fragments(mol, count_labels, args.frequency)

# Preproocess 4
rev_labelmap_dict, deg = revise_maps(labelmap_dict)

# Preproocess 5
labels = []
for k in rev_labelmap_dict.keys():
    for l in rev_labelmap_dict[k]:
        label=[]
        label.append(k)
        label.append(l)
        if label not in labels:
            labels.append(label)

# Preprocess 6
g, sub_tree, root_answer, l_ans_list, b_ans_list = make_graph(mapidxs, labelmap_dict, rev_labelmap_dict ,labels, bondtypes)

# Preprocess 9
mol_ecfp = make_ecfp3D(smiles, args.fpbit, args.radius)
mol_ecfp = np.asarray(mol_ecfp)
mol_ecfp = torch.from_numpy(mol_ecfp).float()
mol_ecfp = mol_ecfp.unsqueeze(0)


g = g.to(torch.device(args.device))
n = g.number_of_nodes()
h = torch.zeros((n, args.h_size)).to(args.device)
c = torch.zeros((n, args.h_size)).to(args.device)    
feat = g.ndata['ecfp'].to(args.device)
m_ecfp = mol_ecfp.to(args.device)

#z, dec_smi, _, mol_pred = model(g, None, None, m_ecfp, smiles, None, feat, h, c, None, None, None, None, None)
z, _, _ = model.encoder(g, m_ecfp, smiles, feat, h, c)
pred_value = model.one_prop_pred(z)
print("Prediction Property of {} is {}".format(smiles,pred_value))

Prediction Property of C(CCCC(=O)O)[C@@H]1CCSS1 is [-0.862643]


/home/yddr/miniconda3/envs/venv/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/home/yddr/miniconda3/envs/venv/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  return warnings.warn(message, category=category, stacklevel=1)


In [6]:
smiles = "ClC(Cl)(Cl)F"

# Preproocess 1

mol = Chem.MolFromSmiles(smiles)
Chem.Kekulize(mol)
mol = Chem.RemoveHs(mol)

# Preproocess 2
cl, frag = count_fragments(mol)
count_labels = Counter(cl)

# Preproocess 3
mapidxs, labelmap_dict, bondtypes, max_mapnums, frag = find_fragments(mol, count_labels, args.frequency)

# Preproocess 4
rev_labelmap_dict, deg = revise_maps(labelmap_dict)

# Preproocess 5
labels = []
for k in rev_labelmap_dict.keys():
    for l in rev_labelmap_dict[k]:
        label=[]
        label.append(k)
        label.append(l)
        if label not in labels:
            labels.append(label)

# Preprocess 6
g, sub_tree, root_answer, l_ans_list, b_ans_list = make_graph(mapidxs, labelmap_dict, rev_labelmap_dict ,labels, bondtypes)

# Preprocess 9
mol_ecfp = make_ecfp3D(smiles, args.fpbit, args.radius)
mol_ecfp = np.asarray(mol_ecfp)
mol_ecfp = torch.from_numpy(mol_ecfp).float()
mol_ecfp = mol_ecfp.unsqueeze(0)


g = g.to(torch.device(args.device))
n = g.number_of_nodes()
h = torch.zeros((n, args.h_size)).to(args.device)
c = torch.zeros((n, args.h_size)).to(args.device)    
feat = g.ndata['ecfp'].to(args.device)
m_ecfp = mol_ecfp.to(args.device)

#z, dec_smi, _, mol_pred = model(g, None, None, m_ecfp, smiles, None, feat, h, c, None, None, None, None, None)
z, _, _ = model.encoder(g, m_ecfp, smiles, feat, h, c)
pred_value = model.one_prop_pred(z)
print("Prediction Property of {} is {}".format(smiles,pred_value))

Prediction Property of ClC(Cl)(Cl)F is [-1.9400072]


### **Real ClC(Cl)(Cl)F LUMO property value : -1.8694**